<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/experiment_Logistic_Regression_only_2_classes_%5BLB_%3F%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google.drive and copy dataset to colab linux instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_DIR ='/content/drive/MyDrive/kaggle/fsnlp_hse/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

WORK_DIR='/content/'
EXPERIMENT='logreg_2c'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

In [ ]:
!mkdir {EXPERIMENT_DIR}

# Install and set up Kaggle API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp
!unzip hse-nlp-bootcamp.zip

 69% 41.0M/59.8M [00:00<00:00, 144MB/s] 
100% 59.8M/59.8M [00:00<00:00, 142MB/s]
Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pprint as pp
import os


# import re
# from pymorphy2 import MorphAnalyzer
# from functools import lru_cache
# from nltk.corpus import stopwords

# from multiprocessing import Pool
# from tqdm import tqdm

import nltk
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from joblib import dump, load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Configuration

In [ ]:
class Config:
  seed = 12536271
  samples_num = 10000
  experiment = EXPERIMENT
  test_size = 0.1

# Main pipeline starts here

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(Config.seed)

## Load and process train data

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['bank_id'] = df['bank'].apply(lambda x: banks_list.index(x))
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df.head()

,bank,feeds,grades,date,bank_id
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3


In [ ]:
len(df)

51476

In [ ]:
df['bin_grades'] = df.grades.apply(lambda x: 1 if x in (1, 2, 3) else 5)
df

,bank,feeds,grades,date,bank_id,bin_grades
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0,1
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4,1
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7,5
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3,1
...,...,...,...,...,...,...
74995,trust,Три года длится эпопея с закрытием кредитной к...,1,2016-10-26 11:03:00,23,1
74996,fk_otkritie,"Являлся клиентом Бинбанка, пока после слияния ...",2,2020-02-01 21:07:00,1,1
74997,sberbank,8 мая 2016 г. мною вносились деньги на банковс...,1,2016-06-14 16:03:00,12,1
74998,uralsib,Здравствуйте! Сегодня 21.04.2020 захожу в моби...,3,2020-04-21 08:24:00,32,1


In [ ]:
subset = df.sample(Config.samples_num, random_state=Config.seed)
subset['text'] = subset.feeds.apply(lambda x: x.lower())
subset.reset_index(drop=True, inplace=True)

## Split data on train and test data

In [ ]:
if Config.test_size > 0.0:
  x_train, x_test, y_train, y_test = train_test_split(subset[['bank_id','text']], subset['bin_grades'], test_size=Config.test_size, random_state=Config.seed)
else:
  x_train, y_train, x_test, y_test = subset[['bank_id','text']], subset['bin_grades'], None, None

In [ ]:
len(x_train)

9000

## Set up train pipeline

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(tokenizer=word_tokenize))
            ]))
         ])),
    ('clf', LogisticRegression(random_state=Config.seed,  warm_start=True, max_iter=1000))
])

In [ ]:
pp.pprint(sorted(pipeline.get_params().keys()))

In [ ]:
param_grid = {'features__text_features__vec__max_df': [1.0],
              'features__text_features__vec__min_df': [0.001],
              'features__text_features__vec__ngram_range': [(1, 2)]}

kfold = StratifiedKFold(n_splits=5)
scoring = {'F1': 'f1_micro'}
refit = 'F1'

## Perform GridSearch

In [ ]:
print(f'samples: {Config.samples_num}')
lr_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=None, return_train_score=True, verbose=10)
lr_model.fit(x_train, y_train)
lr_best = lr_model.best_estimator_

samples: 10000
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START features__text_features__vec__max_df=1.0, features__text_features__vec__min_df=0.001, features__text_features__vec__ngram_range=(1, 2)
[CV 1/5; 1/1] END features__text_features__vec__max_df=1.0, features__text_features__vec__min_df=0.001, features__text_features__vec__ngram_range=(1, 2); F1: (train=0.953, test=0.933) total time=  27.6s
[CV 2/5; 1/1] START features__text_features__vec__max_df=1.0, features__text_features__vec__min_df=0.001, features__text_features__vec__ngram_range=(1, 2)
[CV 2/5; 1/1] END features__text_features__vec__max_df=1.0, features__text_features__vec__min_df=0.001, features__text_features__vec__ngram_range=(1, 2); F1: (train=0.952, test=0.927) total time=  26.8s
[CV 3/5; 1/1] START features__text_features__vec__max_df=1.0, features__text_features__vec__min_df=0.001, features__text_features__vec__ngram_range=(1, 2)
[CV 3/5; 1/1] END features__text_features__vec__max_df=

## Test best model

In [ ]:
if Config.test_size > 0.0:
  pred = lr_best.predict(x_test)
  print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.93      1.00      0.96       714
           5       1.00      0.81      0.89       286

    accuracy                           0.94      1000
   macro avg       0.96      0.90      0.93      1000
weighted avg       0.95      0.94      0.94      1000



In [ ]:
if Config.test_size > 0.0:
  print(f1_score(y_test, pred, average='micro'))

0.945


In [ ]:
lr_model.best_params_

{'features__text_features__vec__max_df': 1.0,
 'features__text_features__vec__min_df': 0.001,
 'features__text_features__vec__ngram_range': (1, 2)}

In [ ]:
lr_model.cv_results_

{'mean_fit_time': array([23.20170979]),
 'std_fit_time': array([0.47421356]),
 'mean_score_time': array([4.09983582]),
 'std_score_time': array([0.09506447]),
 'param_features__text_features__vec__max_df': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__min_df': masked_array(data=[0.001],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__ngram_range': masked_array(data=[(1, 2)],
              mask=[False],
        fill_value='?',
             dtype=object),
 'params': [{'features__text_features__vec__max_df': 1.0,
   'features__text_features__vec__min_df': 0.001,
   'features__text_features__vec__ngram_range': (1, 2)}],
 'split0_test_F1': array([0.93277778]),
 'split1_test_F1': array([0.92722222]),
 'split2_test_F1': array([0.92944444]),
 'split3_test_F1': array([0.93277778]),
 'split4_test_F1': array([0.92444444]),
 'mean_te

In [ ]:
lr_model.best_score_

0.9293333333333333

In [ ]:
cv_tag = f'[cv_{lr_model.best_score_:0.4f}]'

In [ ]:
cv_tag

'[cv_0.9293]'

## Inference (calculating submission) 

In [ ]:
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
test['text'] = test.feeds.apply(lambda x: x.lower())
test['bank_id'] = test['bank'].apply(lambda x: banks_list.index(x))
test = test[['bank_id', 'text']]
test_pred = lr_best.predict(test)
probs = lr_best.predict_proba(test)
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})
sub

,inds,grades
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


# Save results to files

In [ ]:
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'
model_filename = 'model_'+suffix+'.joblib'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs)
dump(lr_best, model_filename) 

['model_logreg_2c_[sn_10000]_[sd_12536271][cv_0.9293].joblib']

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}
!cp {model_filename} {EXPERIMENT_DIR}

# Submit predictions

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:01<00:00, 64.7kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

# Submit CV ~ 0.79 - best public LB 0.79651